In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import random

# Set random seed for reproducibility
random.seed(42)

# Function to perform image augmentation
def augment_images(input_folder, output_folder, num_images):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all images in the input folder
    image_list = os.listdir(input_folder)

    # Calculate the number of images needed for each type of augmentation
    num_images_per_augmentation = (num_images - len(image_list)) // 4

    # Perform augmentation for each image
    for image_name in tqdm(image_list, desc=f'Copying original images to {output_folder}'):
        # Load the image
        image_path = os.path.join(input_folder, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Save the original image
        cv2.imwrite(os.path.join(output_folder, f"original_{image_name}"), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Create an ImageDataGenerator with specified augmentations
    datagen = ImageDataGenerator()

    # Generate augmented images with one type of augmentation
    for i in range(4):
        for j in tqdm(range(num_images_per_augmentation), desc=f'Augmenting {input_folder}'):
            # Load the image
            image_name = random.choice(image_list)
            image_path = os.path.join(input_folder, image_name)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            augmented_img = datagen.apply_transform(img, transform_parameters={'theta': random.uniform(-30, 30)}) if i == 0 else \
                             datagen.apply_transform(img, transform_parameters={'flip_horizontal': True}) if i == 1 else \
                             datagen.apply_transform(img, transform_parameters={'flip_vertical': True}) if i == 2 else \
                             datagen.apply_transform(img, transform_parameters={'channel_shift_intensity': random.uniform(0, 50)}) if i == 3 else None
            
            # Convert the image to 8-bit before saving
            augmented_img = augmented_img.astype(np.uint8)

            # Save the augmented image
            cv2.imwrite(os.path.join(output_folder, f"aug_{i+1}_{j+1}_{image_name}"), cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

# Define input and output folders
input_folder = "Data/train"
output_folder = "new_train"

# Generate 10,000 images for each subfolder
for folder in os.listdir(input_folder):
    input_folder_path = os.path.join(input_folder, folder)
    output_folder_path = os.path.join(output_folder, folder)
    augment_images(input_folder_path, output_folder_path, num_images=10000)
